In [1]:
import numpy as np
import torch
from transformers import AutoTokenizer

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
"""
CHECK YOUR LABEL ENCODED

label_encoding = {"sadness":0,
                  "fear":1,
                  "disgust":2,
                  "neutral":3,
                  "happiness":4,
                  "angry":5,
                  "surprise":6}
"""

label_decoding = {0:"sadness",
                  1:"fear",
                  2:"disgust",
                  3:"neutral",
                  4:"happiness",
                  5:"angry",
                  6:"surprise"}

load test text file

In [3]:
f = open('test_file.txt','r',encoding='UTF8')

load tokenizer

In [4]:
TOKENIZER_NAME = "monologg/koelectra-base-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)

load model

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [6]:
model = torch.load('koelectra_imbalanced.pt', map_location=device)

In [7]:
def sentence_predict(sent):
    # set my model max len 
    MAX_LEN = 64
    
    model.eval()

    tokenized_sent = tokenizer(
        sent,
        return_tensors="pt",
        truncation=True,
        add_special_tokens=True,
        max_length=MAX_LEN
    )

    tokenized_sent.to(device)

    with torch.no_grad():
        outputs = model(
                input_ids = tokenized_sent["input_ids"],
                attention_mask=tokenized_sent["attention_mask"],
                token_type_ids=tokenized_sent["token_type_ids"]
        )

    logits = outputs[0]
    logits = logits.detach().cpu()
    result = logits.argmax(-1)
    
    return label_decoding[np.array(result)[0]]

In [8]:
a = 0
for i in f.readlines():
    answer, sentence = i.split(":")
    sentence = sentence.rstrip("\n")

    print("#######################################")
    print(f'sentence   : ', sentence)
    print(f'answer     : ', answer)
    print(f'prediction : ', sentence_predict(sentence))

#######################################
sentence   :  너 덕분에 할수 있었어 고맙다ㅠㅠㅠ
answer     :  happiness
prediction :  happiness
#######################################
sentence   :  엥 진짜로 ?? 심각한데..;
answer     :  surprise
prediction :  surprise
#######################################
sentence   :  왜 자꾸 하지 말란 짓을 골라서 하냐.. 뭐하자는거임?
answer     :  angry
prediction :  angry
#######################################
sentence   :  3시까지 만나자
answer     :  neutral
prediction :  neutral
#######################################
sentence   :  갑자기 무섭게 왜그래.. 
answer     :  fear
prediction :  fear
#######################################
sentence   :  만나서 얘기해 힘들다.. 이제
answer     :  sadness or angry
prediction :  sadness
#######################################
sentence   :  이번에는 회의 참석 못할것 같아 미안,,
answer     :  neutral and sadness
prediction :  sadness
#######################################
sentence   :  오키 ~ 내일보자
answer     :  neutral
prediction :  neutral
